In [1]:
import logging

import gym
import torch

import neat.population as pop
import neat.experiments.pole_balancing.config as c
from neat.visualize import draw_net
from neat.phenotype.feed_forward import FeedForwardNet

import time
import optuna

/Users/aidan.horn/opt/anaconda3/envs/EvolvingNeuralNetworks/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-15 01:00:35,216 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2022-12-15 01:00:35,217 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2022-12-15 01:00:35,219 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2022-12-15 01:00:35,220 [MainThread  ] [_tools    ] [DEBUG]  deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2022-12-15 01:00:35,223 [MainThread  ] [_tools    ] [DEBUG]  depr

In [20]:
config = c.PoleBalanceConfig

def objective(trial):

    config.SCALE_ACTIVATION = trial.suggest_float('S_ACT', 1 , 10, log = True)

    config.FITNESS_THRESHOLD = 100000.0

    config.POPULATION_SIZE = trial.suggest_int('POP_SIZE', 5, 200)
    config.NUMBER_OF_GENERATIONS = 100
    config.SPECIATION_THRESHOLD = trial.suggest_float('SPEC_THRESH', 1.0, 5.0)

    config.CONNECTION_MUTATION_RATE = trial.suggest_float('CONN_MUT_RATE', 0.1, 0.9)
    config.CONNECTION_PERTURBATION_RATE = trial.suggest_float('CONN_PERT_RATE', 0.1, 0.9)
    config.ADD_NODE_MUTATION_RATE = trial.suggest_float('ADD_NODE_RATE', 0.01, 0.9, log = True)
    config.ADD_CONNECTION_MUTATION_RATE = trial.suggest_float('ADD_CONN_RATE', 0.1, 0.9)

    config.CROSSOVER_REENABLE_CONNECTION_GENE_RATE = trial.suggest_float('CROSSOVER', 0.1, 0.6)

    # Top percentage of species to be saved before mating
    config.PERCENTAGE_TO_SAVE = 0.80

    neat = pop.Population(config)
    start = time.time()
    solution, generation = neat.run()
    end = time.time()

    return (end - start)



In [21]:
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials=300)

trial = study.best_trial

print('Time to Converge: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))



[I 2022-12-16 11:52:33,748] A new study created in memory with name: no-name-530e9e36-cb8a-447c-bc01-913a92780b60
2022-12-16 11:52:34,583 [MainThread  ] [population] [INFO ]  Finished Generation 1
2022-12-16 11:52:34,584 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 352.0
2022-12-16 11:52:34,584 [MainThread  ] [population] [INFO ]  Best Genome Length 5

2022-12-16 11:52:35,265 [MainThread  ] [population] [INFO ]  Finished Generation 2
2022-12-16 11:52:35,265 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 230.0
2022-12-16 11:52:35,265 [MainThread  ] [population] [INFO ]  Best Genome Length 5

2022-12-16 11:52:36,084 [MainThread  ] [population] [INFO ]  Finished Generation 3
2022-12-16 11:52:36,084 [MainThread  ] [population] [INFO ]  Best Genome Fitness: 244.0
2022-12-16 11:52:36,085 [MainThread  ] [population] [INFO ]  Best Genome Length 5

2022-12-16 11:52:37,466 [MainThread  ] [population] [INFO ]  Finished Generation 4
2022-12-16 11:52:37,466 [MainThread  ] [p

Time to Converge: 10.745908975601196
Best hyperparameters: {'S_ACT': 1.7302942104443129, 'POP_SIZE': 44, 'SPEC_THRESH': 4.10793710931474, 'CONN_MUT_RATE': 0.1427746237438695, 'CONN_PERT_RATE': 0.20019289995792447, 'ADD_NODE_RATE': 0.15674241931159313, 'ADD_CONN_RATE': 0.15627141474707587, 'CROSSOVER': 0.2873862035652746}


In [15]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_slice(study) 
optuna.visualization.plot_contour(study, params=['S_ACT', 'POP_SIZE', 'SPEC_THRESH', 'CONN_MUT_RATE'], target_name='Time to Converge')

/Users/aidan.horn/opt/anaconda3/envs/EvolvingNeuralNetworks/lib/python3.8/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [18]:
optuna.visualization.plot_contour(study, params=['POP_SIZE', 'CONN_MUT_RATE'], target_name='Time to Converge')

In [19]:
optuna.visualization.plot_contour(study, params=['POP_SIZE', 'SPEC_THRESH'], target_name='Time to Converge')